In [ ]:
SQLalchemy



In [1]:
import pandas as pd
import ipywidgets as ipw
from IPython import display
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

In [2]:
#getting data
df=pd.read_csv('shuttle_three_days.csv')

C:\Users\Anis Mousse\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#retrieve_data_criteria(df,'STOP_LOCATION_LATITUDE','SETRA','STOP_LOCATION_LONGITUDE')
#retritrie_data(df,'STOP_LOCATION_LATITUDE','')

In [4]:
# compagny: STOP_LOCATION_LATITUDE                                    
#bus ID STOP_LOCATION_LONGITUDE                                     
#latitude SHUTTLE_COMPANY                                           
#longitude TECH_PROVIDER_NAME   
#date TECH_PROVIDER                                           


In [5]:
#initial data
sf_coor=[37.77397,-122.43129]
zoom=13
circle_list=[]
m = Map(default_tiles=TileLayer(opacity=1.0), center=sf_coor, zoom=zoom)
circle_list=[]

In [6]:
#functions 
def retrieve_data_criteria(data,filter_selected,filter_value):     
    return data[data[filter_selected]==filter_value]       

def retritrie_data(data,filter_selected,defaut_value=None):
    if defaut_value==None:
        return list(set(data[filter_selected]))    
    else:
        return [defaut_value] + list(set(data[filter_selected]))
    
def add_data_to_map(mapp,data,latitude,longitute,radius=30,weight=1,color='#F00',opacity=1.0,fill_opacity=1.0,fill_color='#F00'):
    
    tmp=(100.-progress.value)/data[latitude].count()
    
    for point in zip(data[latitude],data[longitute]):
        circle = Circle(location=point, radius=radius, weight=weight,
            color=color, opacity=opacity, fill_opacity=fill_opacity,
            fill_color=fill_color)
        circle_list.append(circle)
        mapp.add_layer(circle) 
        progress.value+=tmp
    
        
def remove_data_from_map(mapp):
    for circle in circle_list:
        mapp.remove_layer(circle)
        
def load_data_to_map(mapp,bus_company_value,bus_id_value,bus_date_value,bus_color):
    progress.value+=10
    tmp=retrieve_data_criteria(
            retrieve_data_criteria(
                retrieve_data_criteria(df,'STOP_LOCATION_LATITUDE',bus_company_value),
                        'STOP_LOCATION_LONGITUDE',bus_id_value),'TECH_PROVIDER',bus_date_value)
    #print (tmp)
    progress.value+=10
    add_data_to_map(mapp,tmp,'SHUTTLE_COMPANY','TECH_PROVIDER_NAME',color=bus_color)
    
def get_bus_date_by_bus_id_and_bus_company(bus_company_value,bus_id_value,bus_date_wj):
    bus_date_wj.options=retritrie_data(retrieve_data_criteria(
                                        retrieve_data_criteria(df,'STOP_LOCATION_LATITUDE',bus_company_value),
                                        'STOP_LOCATION_LONGITUDE',bus_id_value),
                                     'TECH_PROVIDER')
    
def get_bus_id_by_bus_company(bus_company_value,bus_id_wj):
    bus_id_wj.options=retritrie_data(retrieve_data_criteria(df,'STOP_LOCATION_LATITUDE',bus_company_value),'STOP_LOCATION_LONGITUDE')
    
def init_application(bus_comp_wj,bus_id,bus_date):
    #get all bus company
    bus_comp_wj.options=retritrie_data(df,'STOP_LOCATION_LATITUDE','')


    #print (bus_company.value)

In [7]:

bus_company=ipw.Dropdown(description='Bus Company')
bus_id=ipw.Dropdown(description='Bus ID')
bus_selected_date=ipw.Dropdown(description='Date selected')
bus_color_on_map=ipw.ColorPicker(description='Bus color')
send_button=ipw.Button(description='Update Map')
clear_button=ipw.Button(description='Clear Map')
init_application(bus_company,bus_id,bus_selected_date)
progress=ipw.FloatProgress(min=0.0,max=100.0,value=0,description ='')
contenaire_layer_1=ipw.HBox([bus_company,bus_id,bus_selected_date])
contenaire_layer_2=ipw.HBox([bus_color_on_map,send_button,clear_button])
contenaire=ipw.VBox([contenaire_layer_1,contenaire_layer_2,progress])


def on_bus_comp_select(bus_comp_drop):
    print(bus_company.value)
    get_bus_id_by_bus_company(bus_company.value,bus_id)
    
    #if type(bus_id_drop['new'])== string:
        #get_bus_id_by_bus_company(bus_id_drop['new'],bus_id)
def on_bus_id_select(bus_id_drop):
    get_bus_date_by_bus_id_and_bus_company(bus_company.value,bus_id.value,bus_selected_date)
        
def on_update_map(updatebtt):
    progress.value=0
    load_data_to_map(m,bus_company.value,bus_id.value,bus_selected_date.value,bus_color_on_map.value)

def on_clear_map(clear):
    remove_data_from_map(m)
    
bus_company.observe(on_bus_comp_select)
bus_id.observe(on_bus_id_select)
send_button.on_click(on_update_map)
clear_button.on_click(on_clear_map)
display.display(contenaire)
display.display(m)

VBox(children=(HBox(children=(Dropdown(description='Bus Company', options=('', nan, 'STRC', 'Sprinter', 'CHEVROLET', 'SETRA', 'MCI', 'chevorlet', 'Vanhool', 'CHEVY', 'FORD', 'Temsa', 'VanHool ', 'Ford', 'Vanholl', 'TEMSA', 'Mercedes', 'Freightliner', 'Van Hool', 'VanHool', 'Ford ', 'Prevost', 'GNRC', 'VANHOOL'), value=''), Dropdown(description='Bus ID', options=(), value=None), Dropdown(description='Date selected', options=(), value=None))), HBox(children=(ColorPicker(value='black', description='Bus color'), Button(description='Update Map', style=ButtonStyle()), Button(description='Clear Map', style=ButtonStyle()))), FloatProgress(value=0.0)))

Map(center=[37.77397, -122.43129], layers=(TileLayer(options=['min_zoom', 'tile_size', 'detect_retina', 'opacity', 'attribution', 'max_zoom']),), layout=Layout(align_self='stretch', height='400px'), options=['close_popup_on_click', 'inertia', 'keyboard', 'touch_zoom', 'min_zoom', 'scroll_wheel_zoom', 'attribution_control', 'bounce_at_zoom_limits', 'keyboard_pan_offset', 'world_copy_jump', 'max_zoom', 'tap', 'zoom_animation_threshold', 'double_click_zoom', 'tap_tolerance', 'inertia_deceleration', 'dragging', 'zoom_control', 'box_zoom', 'center', 'zoom_start', 'inertia_max_speed', 'keyboard_zoom_offset', 'zoom'], zoom=13)

In [8]:
import ipyleaflet
ipyleaflet.__version__

'0.4.0'